In [1]:
import json
import httpx
import asyncio
import pandas as pd

from os import path, mkdir
from datetime import datetime

from toolkit.models import Timeframe
from toolkit.exchanges import Binance

In [2]:
binance = Binance()

timeframe = Timeframe.MINUTLY
binance_markets = ["BTCUSDT", "ETHUSDT", "LTCUSDT"]

In [3]:
print("Downloading markets...")
kline_data = []
async with httpx.AsyncClient() as client:
    for symbol in binance_markets:
        kline_data.append(binance.kline(client, symbol, timeframe))
    kline_data = await asyncio.gather(*kline_data)
    kline_data = dict(zip(binance_markets, kline_data))
print("Finished downloading.")

Finished downloading.


In [4]:
data_path = path.join("data")
index_path = path.join(data_path, "_index.json")

index = {}
if path.isfile(index_path):
    with open(index_path, "r") as f:
        index = json.load(f)

if len(index) == 0:
    for symbol, trades in kline_data.items():
        df = pd.DataFrame.from_dict(trades)
        index[symbol] = df.iloc[-1]["time"].isoformat()

        df.to_csv(path.join(data_path, f"{symbol}_Kline_{timeframe.value}"), index=False)
    with open(index_path, "w+") as f:
        json.dump(index, f)
else:
    for symbol, trades in kline_data.items():
        df = pd.DataFrame.from_dict(trades)
        new = df.copy()
        
        if symbol in index:
            new = df[df["time"] > index[symbol]]

        if not new.empty:
            new.to_csv(path.join(data_path, f"{symbol}_Kline_{timeframe.value}"), mode="a", index=False, header=False)
            index[symbol] = new.iloc[-1]["time"].isoformat()
    with open(index_path, "w") as f:
        json.dump(index, f)